In [2]:
import numpy as np
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep

In [3]:
requests.get('http://ies.fsv.cuni.cz/robots.txt')

<Response [404]>

In [5]:
print(requests.get('http://ies.fsv.cuni.cz/robots.txt').text)

<!doctype html>
<!--[if lte IE 7]> <html class="no-js ie7" lang="cs"> <![endif]-->
<!--[if IE 8]>     <html class="no-js ie8" lang="cs"> <![endif]-->
<!--[if IE 9]>     <html class="no-js ie9" lang="cs"> <![endif]-->
<!--[if !IE]><!--> <html class="no-js" lang="cs">             <!--<![endif]-->
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta http-equiv="Content-Language" content="cs">
        <meta name="robots" content="INDEX,FOLLOW">
        <meta name="document-distribution" content="Global">
        <meta name="cache-control" content="Public">
        <meta name="generator" content="eControl by Motion Media">
        <meta name="author" content="IES - Institute of economics studies">
        <meta name="copyright" content="IES - Institute of economics studies">
        <meta name="description" content="IES - Institut ekonomickÃ½ch studiÃ­">
        <meta name="keywords" content="IES,institut,ekonomickÃ½ch,studiÃ­

In [21]:
Instance_IES = External('https://ies.fsv.cuni.cz/en/node/49')
Instance_IES.saveDFs().head(5)

,Name,Position,Phone,E-mail
ba,Mgr. Lei Ba,Teaching Fellow,not available,balei [DOT] kayser [AT] gmail [DOT] com
babiak,Mykola Babiak MA,Teaching Fellow,not available,balei [DOT] kayser [AT] gmail [DOT] com
bajura,doc. JUDr. Mgr. Jan Bajura Ph.D.,Teaching Fellow,not available,balei [DOT] kayser [AT] gmail [DOT] com
balcar,Ing. Petr Balcar MSBE,Teaching Fellow,not available,balei [DOT] kayser [AT] gmail [DOT] com
barta,RNDr. Tomáš Bárta Ph.D.,Teaching Fellow,not available,balei [DOT] kayser [AT] gmail [DOT] com


In [20]:
Instance_Current = External("https://ies.fsv.cuni.cz/en/node/48")
Instance_Current.saveDFs().head(5)

,Name,Position,Phone,E-mail
barunik,doc. PhDr. Jozef Baruník Ph.D.,Associate Professor,+420(776)259273,barunik [AT] fsv [DOT] cuni [DOT] cz
bauerm,doc. PhDr. Michal Bauer Ph.D.,Associate Professor,+420(776)259273,barunik [AT] fsv [DOT] cuni [DOT] cz
baxajaromir,PhDr. Jaromír Baxa Ph.D.,Associate Professor,+420(776)259273,barunik [AT] fsv [DOT] cuni [DOT] cz
bertoli,"doc. Paola Bertoli M.A., MSc., Ph.D.",Associate Professor,+420(776)259273,barunik [AT] fsv [DOT] cuni [DOT] cz
antosova,PhDr. Lucie Bryndová,Associate Professor,+420(776)259273,barunik [AT] fsv [DOT] cuni [DOT] cz


In [17]:
class External:
    '''
    Downloader manager for External lecturers from IES
    
    It contains methods for collection of links, downloading itself and storing results
    '''
    def __init__(self, link):
        '''
        Creates the External object with self.links that store links to be downloaded
        to store individual people teaching at IES
        '''
        self.link = link
        
    def getSoup(self):
         '''Downloads the link of the specified webpage for people and saves it to self.link.'''
        r = requests.get(self.link)
        r.encoding = "UTF-8"
        soup = BeautifulSoup(r.text, "lxml")
        return soup
    
    def getAllLinks(self):
        '''Downloads all person links in the specified webpage.'''
        soup2 = self.getSoup().findAll("td", {"class": "peopleTableCellName"})
        links2 = ["https://ies.fsv.cuni.cz" + s.find("a")["href"] for s in soup2]
        return links2
    
    def getName(self):
        '''Extract the name of lecturer.'''
        r2 = []
        for li in self.getAllLinks():
            r3 = requests.get(li)
            r3.encoding = "UTF-8"
            r4 = BeautifulSoup(r3.text, "lxml").find("h2").text
            r2.append(r4)
            
        return r2
    
    def getID(self):
        '''Extract the ID of lecturer.'''
           # 'https://ies.fsv.cuni.cz/en/staff/zeleny'
        IDs = [id.split('/')[-1] for id in self.getAllLinks()]
        return IDs
    
    def getNextSiblingOfStrong(self, characteristic):
        '''
        Function for the extraction of characteristics for lecturers
        These are then specified in saveDFs function
        '''
        si = []
        for li in self.getAllLinks():
            si1 = requests.get(li)
            si1.encoding = "UTF-8"
            si2 = BeautifulSoup(si1.text, "lxml").find("strong", text = characteristic).next_sibling.strip()
            si.append(si2)
            
            return si
        
    def saveDFs(self):
        '''Specifying characteristics for lecturers and generate Panda Data Frame.'''
        names = self.getName()
        ids = self.getID()
        pos = self.getNextSiblingOfStrong("Position:")
        phones = self.getNextSiblingOfStrong("Phone:")
        office = self.getNextSiblingOfStrong("Office:")
        mail = self.getNextSiblingOfStrong("Email:")
        df = pd.DataFrame({
            "Name": names, 
            "Position": pos,
            "Phone": phones,
            "E-mail": mail}, index = ids)
        
        return df

        


In [81]:
idx = np.linspace(1, 78, 78)
idx = np.round(idx)
idx = idx.astype(int)
idx

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78])